In [3]:
%load_ext autoreload
%autoreload 2

In [5]:
from food.tools import *
from food.paths import *

import pandas as pd
import numpy as np
import requests
import telebot
from food.psql import *

# !nbdev_build_lib
from food.classify_image import *

from telebot import types


2022-04-05 00:48:50,889 (__init__.py:688 MainThread) ERROR - TeleBot: "A request to the Telegram API was unsuccessful. Error code: 400. Description: Bad Request: message text is empty"
2022-04-05 00:49:21,797 (__init__.py:688 MainThread) ERROR - TeleBot: "A request to the Telegram API was unsuccessful. Error code: 400. Description: Bad Request: message text is empty"
2022-04-05 00:49:53,701 (__init__.py:621 MainThread) ERROR - TeleBot: "Infinity polling: polling exited"
2022-04-05 00:49:53,702 (__init__.py:623 MainThread) ERROR - TeleBot: "Break infinity polling"


In [6]:

token = "5091011572:AAG4NfkC_zZjcsaAFkwLm4ZXOvhEqyLpQhY"

m = None

bot = telebot.TeleBot(token)
bot.dish = None


@bot.message_handler(commands=['start', 'help'])
def send_welcome(message):
	bot.reply_to(message, "Howdy, how are you doing?")

@bot.message_handler(commands=['cancel'])
def send_cancel(message):
	markup = types.ReplyKeyboardRemove(selective=False)
	bot.reply_to(message, f"", reply_markup=markup)


@bot.message_handler(content_types=['photo'])
def calssify_image(message):

	bot.send_chat_action(message.chat.id, 'typing') #does not seem to work
	global m
	m = message
	image_url = bot.get_file_url(message.photo[3].file_id)
	bot.dish = search_image(url=image_url)
	bot.dish['image_url'] = image_url
	bot.dish['user_id']= message.from_user.id

	plot_numtients = bot.dish[['energy','protein','carb','fat']].reset_index(drop=True)
	plot_numtients.index = ['']

	bot.reply_to(message, bot.dish['description'].iloc[0])
	bot.reply_to(message, plot_numtients.to_string())

	bot.measures = pd.read_sql(f"select portion_description,gram_weight from portions where food_id = {bot.dish['id'].iloc[0]}",engine)

	markup = types.ReplyKeyboardMarkup(row_width=2)

	cancel = types.KeyboardButton('/cancel')
	[markup.add(p) for p in bot.measures['portion_description'].tolist()]
	markup.add(cancel)

	bot.send_message(message.chat.id, "Choose amount:", reply_markup=markup)


	




@bot.message_handler(regexp="[0-9]+")
def handle_grams(message):
	global m
	m = message
	if hasattr(bot,'dish'):  
		grams = bot.measures[bot.measures['portion_description'] == message.text]['gram_weight'].iloc[0]
		bot.dish[['energy','protein','carb','fat']]/100*grams
		bot.dish['grams']=grams
		bot.dish['measure_selected'] = message.text
		bot.dish = bot.dish.rename(columns = {'id':'food_id'})
		bot.dish['timestamp']=pd.Timestamp.utcnow() #should be user local time


		bot.reply_to(message, bot.dish)
		bot.dish.to_sql('dishes',engine,if_exists='append',index=False)

		last_day = pd.read_sql(f"select sum(energy) from dishes where user_id = {message.from_user.id} and timestamp > now() - interval '24 hours';",engine)

		markup = types.ReplyKeyboardRemove(selective=False)
		bot.reply_to(message, f"you have consumed {last_day.iloc[0]['sum']} kcall in the last 24 hours", reply_markup=markup)

bot.infinity_polling()

2022-04-05 00:50:14,902 (__init__.py:688 MainThread) ERROR - TeleBot: "A request to the Telegram API was unsuccessful. Error code: 400. Description: Bad Request: message text is empty"
2022-04-05 00:50:24,920 (__init__.py:688 MainThread) ERROR - TeleBot: "A request to the Telegram API was unsuccessful. Error code: 400. Description: Bad Request: message text is empty"
2022-04-05 00:50:47,992 (__init__.py:621 MainThread) ERROR - TeleBot: "Infinity polling: polling exited"
2022-04-05 00:50:47,992 (__init__.py:623 MainThread) ERROR - TeleBot: "Break infinity polling"


In [12]:
to_plot.index=['']

In [13]:
to_plot

,description,energy,measure_selected,grams
,"Grapes, raw",69.0,1 cup,150.0


In [20]:
message = m

In [25]:
last_day = pd.read_sql(f"select sum(energy) from dishes where user_id = {message.from_user.id} and timestamp > now() - interval '24 hours';",engine)
last_day.iloc[0]['sum']

In [14]:
plot_numtients = bot.dish[['energy','protein','carb','fat']].reset_index(drop=True)
plot_numtients.index = ['']
print(plot_numtients.to_string())

In [6]:
message = m

In [7]:
image_url = bot.get_file_url(message.photo[3].file_id)

bot.dish = search_image(url=image_url)
bot.dish['image_url'] = image_url
bot.dish['user_id']= message.from_user.id

In [8]:
bot.dish

,id,description,category,energy,protein,carb,fat,clip,score,image_url,user_id
4483,1101970,"Pizza, cheese, from restaurant or fast food, N...",Pizza,266.0,11.39,33.33,9.69,"[0.009587851, 0.02125338, 0.03126051, -0.01058...",0.242496,https://api.telegram.org/file/bot5091011572:AA...,78882798


In [9]:
bot.dish.to_string()

'           id                                                          description category  energy  protein   carb   fat                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                              

In [7]:
bot.dish.to_frame().T

,energy_kcal,proteins,fat,carbohydrates,ids,image_url,user_id,grams,timestamp
0,405.57,5.04,14.12,64.1,"[336307, 401550, 588494, 655972, 806666, 1091422]",https://api.telegram.org/file/bot5091011572:AA...,78882798,30,2022-03-30 21:40:07.337149+00:00


In [3]:
bot.dish.to_frame().T

,energy_kcal,proteins,fat,carbohydrates,ids,image_url,user_id,grams,timestamp
0,370.0,5.87,11.57,63.87,"[401550, 447468, 655972, 806666, 1111863, 2680...",https://api.telegram.org/file/bot5091011572:AA...,78882798,200,2022-03-30 21:31:36.950219+00:00


In [15]:
message = m

In [16]:
int(message.text)

TypeError: int() argument must be a string, a bytes-like object or a number, not 'NoneType'

In [14]:
bot.dish

energy_kcal                                                 476.33
proteins                                                      14.2
fat                                                          20.43
carbohydrates                                                38.99
image_url        https://api.telegram.org/file/bot5091011572:AA...
user_id                                                   78882798
dtype: object

In [89]:
bot.dish/100*int(message.text)

energy_kcal      38.266
proteins          0.106
fat               0.704
carbohydrates     6.614
grams             4.000
dtype: float64

In [88]:
bot.dish

energy_kcal      191.33
proteins           0.53
fat                3.52
carbohydrates     33.07
grams             20.00
dtype: float64

In [73]:
bot.dish/100*250

energy_kcal      151.675
proteins           4.400
fat                2.900
carbohydrates     13.850
dtype: float64

In [74]:
bot.dish

energy_kcal      60.67
proteins          1.76
fat               1.16
carbohydrates     5.54
dtype: float64

In [61]:
message.chat.id

78882798

In [30]:
file_url = bot.get_file_url(message.photo[3].file_id)

In [31]:
df = get_calories(url=file_url)

In [52]:
df.index = df.index.str.replace('_100g','')

In [53]:
t = df.to_frame('per 100g').to_string()

In [54]:
print(t)

               per 100g
energy_kcal      210.33
proteins           4.93
fat               10.47
carbohydrates     23.33


In [55]:
bot.reply_to(message, t)

In [17]:
file_info

'https://api.telegram.org/file/bot5091011572:AAG4NfkC_zZjcsaAFkwLm4ZXOvhEqyLpQhY/photos/file_89.jpg'

In [14]:
file_info.file_path

'photos/file_89.jpg'

In [15]:
f'https://api.telegram.org/file/bot{token}/{file_info.file_path}'

'https://api.telegram.org/file/bot5091011572:AAG4NfkC_zZjcsaAFkwLm4ZXOvhEqyLpQhY/photos/file_89.jpg'

In [10]:
m.chat.id

78882798